# Assignmnent 1 DRM

#### by Dag Stabell Storhaug, Kristian Wang Høiem, Paul Joakim Andreassen, og Espen Sønneland

## Abstract



## The Code

In [1]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

## Task 1

In [2]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

In [1]:
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

hours = 24
peak_hour_from = 17
peak_hour_to = 20
num_appl = 3 # Number of appliances

dishwasher     = np.zeros(num_appl*hours) #[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
washingmachine = np.zeros(num_appl*hours)  #[0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0]
EV             = np.zeros(num_appl*hours) #[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]

dishwasher[:24] = 1
washingmachine[24:48] = 1
EV[48:] = 1

TotalEnergyConsumption_dishwasher = 1.44 # kWh
TotalEnergyConsumption_washingmacine = 1.94  # kWh
TotalEnergyConsumption_EV = 9.9 # kWh


maxPowerUsage_dish = 1.8 # kW brukt per time
maxPowerUsage_wash = 0.5 # kW brukt per time
maxPowerUsage_EV = 3.6 # kW brukt per time 
maxPowerusage_total = 10 # kW brukt per time

price = np.ones(hours)*0.5
price[peak_hour_from:peak_hour_to] = 1 # peak-hour 5pm-8pm

A_eq = np.array([                        # Hvilke dingser som brukes i likhetsbegrensningen
                dishwasher,
                washingmachine,
                EV
])

b_eq = np.array([TotalEnergyConsumption_dishwasher,      # Summen av likhetsbegresningen
                 TotalEnergyConsumption_washingmacine,
                 TotalEnergyConsumption_EV
])


A_mul = np.zeros((hours,num_appl*hours))
for i in range(A_mul.shape[0]):
    A_mul[i,i::hours] = 1
A_one = np.eye(num_appl*hours)    
A_ub = np.concatenate((A_one,A_mul),axis=0)  # Hvilket timesobjekt begrensningen hører til 


b_one_ub = np.array( [maxPowerUsage_dish]*hours    # Begrensning
                    +[maxPowerUsage_wash]*hours
                    +[maxPowerUsage_EV]*hours
)
b_max_ub = np.array([maxPowerusage_total]*hours)

b_ub = np.concatenate((b_one_ub,b_max_ub)) 

c = np.array([])  #np.array([p1,p2,p3,p4,p1,p2,p3,p4]) Coefficients of the linear objective function to be minimized
for _ in range(num_appl):
    c = np.append(price,c)

res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=(0, None))
print('Optimal value:', res.fun, '\nX:\n', res.x.reshape(num_appl,hours))


Optimal value: 6.640000000000001 
X:
 [[1.44 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.5  0.5  0.5  0.44 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [3.6  3.6  2.7  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]]


In [2]:
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

def opt(name=None, consumption=0.0, maxPower=0.0, interval=None, 
        duration=1, starttime=None, price=0.0, maxPower_total=10):
    '''
    Optimises the daily cost related to energy consumption for a house with x appliances.
    
    Returns the optimal value and a matrix [number of appliances x hours of the day] consisting of hourly energy values, with rows 
    and columns corresponding to objects and hours of the day, respectively.
    
    ---------------------------------
    Linear programming of the problem:
    
    minimise f(x) = c1x1_1 + c2x1_2 + c3x1_3 +..+ cnx1_n + c1x2_1 + c2x2_2 + c3x2_3 +..+ cnx2_n +..+ c1xm_1 + c2xm_2 + c3xm_3 +..+ cnxm_n
    s.a.
    
    x1_1 + x1_2 +x1_3 +..+ x1_n = consumption1
    x2_1 + x2_2 +x2_3 +..+ x2_n = consumption2
                        .
                        .
    xm_1 + xm_2 + xm_3 +..+ xm_n = consumptionm
    
    x1_1 <= maxPower1
    x1_2 <= maxPower1
    x1_3 <= maxPower1
            .
    x1_n <= maxPower1
    x2_1 <= maxPower2
    x2_2 <= maxPower2
    x2_3 <= maxPower2
            .
    x2_n <= maxPower2
    xm_1 <= maxPowerm
    xm_2 <= maxPowerm
    xm_3 <= maxPowerm
            .
    xm_n <= maxPowerm
    
    x1_1 + x2_1 +..+ xm_1 <= maxPower_total
    x1_2 + x2_2 +..+ xm_2 <= maxPower_total
                        .
    x1_n + x2_n +..+ xm_n <= maxPower_total
    
    x >= 0
    
    
    '''
    
    # Initialising 
    hours = 24
    peak_hour_from = 17
    peak_hour_to = 20
    num_appl = len(name) # Number of appliances

    #maxPowerusage_total = 10 # kW brukt per time

    price = np.ones(hours)*0.5
    price[peak_hour_from:peak_hour_to] = 1 # peak-hour 5pm-8pm

    # Initialising hourly appliance consumption matrix used for optimisation
    A_eq = np.zeros((num_appl, num_appl*hours))
    
    # Assigns which element should correspond to a given appliance keeping the order.
    for i in range(num_appl):
        A_eq[i,i*hours:(i+1)*hours] = 1
      
    # The sum of equality constraints  
    b_eq = consumption
    
    # Which time object the constraints belong to.
    A_mul = np.zeros((hours,num_appl*hours))
    for i in range(A_mul.shape[0]):
        A_mul[i,i::hours] = 1
    A_one = np.eye(num_appl*hours)    
    A_ub = np.concatenate((A_one,A_mul),axis=0)

    # Constraints
    b_one_ub = []
    for i in range(num_appl):
        b_one_ub += [maxPower[i]]*hours
    b_one_ub = np.array(b_one_ub)
    b_max_ub = np.array([maxPower_total]*hours)
    b_ub = np.concatenate((b_one_ub,b_max_ub)) 
    
    # Price coefficents of the linear objective function to be minimized
    c = np.array([])
    for _ in range(num_appl):
        c = np.append(price,c)

    res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=(0, None))
    print('Optimal value:', res.fun, '\nX:\n', res.x.reshape(num_appl,hours))
    
    return res.fun, res.x.reshape(num_appl,hours)
    

In [3]:
import pandas as pd
import numpy as np
       
        
class House():
    def __init__(self):
        pass
        
    def cnstr(self):
    
        Apps.construct()
        apps = [] # inneholder alle navn til apps
        pd.DataFrame(data=[],index=apps, columns=[])
    
    def opt(self):
        pass
#
  

def random_float(a=0, b=0):
    '''
    Takes two floating point numbers and returns a floating point number between these numbers. 
    '''
    return float((b-a)*np.random.random_sample(1) + a)

def appliances():  
    '''
    Returns a dataframe with appliances as indexes and properties such as energy consumption as columns.
    '''
    Appliances = {
        'Heating':{
             'consumption': random_float(6.4,9.6), 
             'maxPower':0,
             'interval': 0, 
             'duration': 0, 
             'starttime': 0, 
             'shiftable': 0
        },
        'Refrigerator_Freezer':{
            'consumption': random_float(1.32,3.96), 
            'maxPower':0,
            'interval': 0, 
            'duration': 0, 
            'starttime': 0, 
            'shiftable': 0
        },
        'Stove':{
            'consumpotion': 3.9,
            'maxPower':0,
            'interval': 0, 
            'duration':0 , 
            'starttime':0 , 
            'shiftable': 0
        },
        'Dishwasher':{
            'consumption': 1.44, 
            'maxPower': 1.8,
            'interval': 0, 
            'duration': 0 , 
            'starttime': 0, 
            'shiftable': 1
        },
        'LaundryMachine':{
            'consumption': 1.94,
            'maxPower' : 0.5,
            'interval': 0, 
            'duration': 0, 
            'starttime': 0, 
            'shiftable': 1
        },
        'EV':{
            'consumption': 9.9,
            'maxPower': 3.6,
            'interval': 0, 
            'duration': 0, 
            'starttime': 0, 
            'shiftable': 1
        },
                #'ClothDryer',
                #'Tv',
                #'Computer',
                #'CoffeeMaker',
                #'CeilingFan',
                #'HairDryer',
                #'Toaster',
                #'Microwave',
                #'Router',
                #'CellphoneCharge',
                #'CliothIron',
                #'SeparateFreezer'
    }
    
    Appliances_df = pd.DataFrame.from_dict(data=Appliances, orient='index')  
    
    return Appliances_df    
    
    
if __name__ == '__main__':
   
    app_df = appliances()
    app_shift = app_df[app_df['shiftable'] == 1]
    consumption = app_shift['consumption'].values
    maxPower = app_shift['maxPower'].values
    interval = app_shift['interval'].values
    duration = app_shift['duration'].values
    starttime = app_shift['starttime'].values
    app_label = app_shift.index
    opt(name=app_label, consumption=consumption, maxPower=maxPower, interval=interval, duration=duration, starttime=starttime)


Optimal value: 6.640000000000001 
X:
 [[1.44 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [3.6  3.6  2.7  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.5  0.5  0.5  0.44 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]]


### Discution

## Task 2

In [3]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

### Discution

## Task 3

In [4]:
"""
Putt inn coden her
"""

'\nPutt inn coden her\n'

### Discution

## Conclution